In [62]:
import os
import cv2
import numpy as np
from sklearn.cluster import MiniBatchKMeans
import glob
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pickle

In [63]:
def labeled_data(path_data,path_image):
    subject=os.listdir(path_data)
    labeled_data=[]
    for i in range(0,len(subject)):
        
        path_subject=path_data+'/'+subject[i]
    
        subject_sequence= os.listdir(path_subject)
   
        for j in range(0,len(subject_sequence)):
            path_sequence=path_subject+'/'+subject_sequence[j]
            emotion=os.listdir(path_sequence)
            if len(emotion)==0:
                continue
            f = open(path_sequence+'/'+emotion[0], 'r')
            emo_tag=f.readlines()
            emo=int(float(emo_tag[0]))
            image_name = emotion[0][0:-12]
            image_path = path_image+'/'+subject[i]+'/'+subject_sequence[j]+'/'+image_name+'.png'
            image_tuple=(image_path,emo)
            labeled_data.append(image_tuple)
    return labeled_data
        

In [65]:
def img_desc_sift(img_path):
    face_cord=image_pre_harr(img_path)
    img = cv2.imread(img_path)
    if len(face_cord) == 0 :
        return []
    (x,y,w,h) = face_cord[0]
    img=img[y:y+h, x:x+w]
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(gray,None)
    #print 'image:',img_path,'# descriptors:',des.shape
    return des

In [66]:
def image_pre_harr(img_path):
    face_cascade = cv2.CascadeClassifier('/home/atrij/opencv/data/haarcascades/haarcascade_frontalface_default.xml')
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    return faces

In [68]:
def decs_dataset(labeled_data):
    des=[]
    Y=[]
    print 'computing descriptors:', len(labeled_data)
    for data in labeled_data:
        #print 'image number',i
        #print data[0]
        des_img=img_desc_sift(data[0])
        if len(des_img)==0:
            continue
        des.append(des_img)
        label=data[1]
        Y.append(label)
    #des_stack_train=[desc for img in des_train for desc in img]
    return des,Y

In [69]:
def save_descriptor_label(filename_des,filename_label,labeled_data):
    if os.path.exists(filename_des) & os.path.exists(filename_label):
        return
    else:
        des,y=decs_dataset(labeled_data)
        fileObject = open(filename_des,'wb')
        pickle.dump(des,fileObject)
        fileObject.close()
        fileObject1 = open(filename_label,'wb')
        pickle.dump(y,fileObject1)
        fileObject1.close()
        return
        

In [70]:
path_data="/home/atrij/artificial_intelligence/CK+/Emotion"
path_image="/home/atrij/artificial_intelligence/CK+/cohn-kanade-images"

ld= labeled_data(path_data,path_image)

In [71]:
filename_des='img_des'
filename_label='img_label'

In [72]:
def train_test_val_split_idxs(total_rows, percent_test, percent_val):
    """
    Get indexes for training, test, and validation rows, given a total number of rows.
    Assumes indexes are sequential integers starting at 0: eg [0,1,2,3,...N]

    Returns:
    --------
    training_idxs, test_idxs, val_idxs
        Both lists of integers
    """
    if percent_test + percent_val >= 1.0:
        raise ValueError('percent_test and percent_val must sum to less than 1.0')

    row_range = range(total_rows)

    no_test_rows = int(total_rows*(percent_test))
    test_idxs = np.random.choice(row_range, size=no_test_rows, replace=False)
    # remove test indexes
    row_range = [idx for idx in row_range if idx not in test_idxs]

    no_val_rows = int(total_rows*(percent_val))
    val_idxs = np.random.choice(row_range, size=no_val_rows, replace=False)
    # remove validation indexes
    training_idxs = [idx for idx in row_range if idx not in val_idxs]

    #print 'Train-test-val split: %i training rows, %i test rows, %i validation rows' % (len(training_idxs), len(test_idxs), len(val_idxs))

    return training_idxs, test_idxs, val_idxs

In [73]:
def cluster_features(img_descs,cluster_model):
    """
    Cluster the training features using the cluster_model
    and convert each set of descriptors in img_descs
    to a Visual Bag of Words histogram.

    Parameters:
    -----------
    X : list of lists of SIFT descriptors (img_descs)

    training_idxs : array/list of integers
        Indicies for the training rows in img_descs

    cluster_model : clustering model (eg KMeans from scikit-learn)
        The model used to cluster the SIFT features

    Returns:
    --------
    cluster_model :
    cluster_model has been fit to the training set
    """
    n_clusters = cluster_model.n_clusters
    des_stack_train=[desc for img in img_descs for desc in img]
    all_train_descriptors = np.array(des_stack_train)

    if all_train_descriptors.shape[1] != 128:
        raise ValueError('Expected SIFT descriptors to have 128 features, got', all_train_descriptors.shape[1])

    print '%i descriptors before clustering' % all_train_descriptors.shape[0]

    # Cluster descriptors to get codebook
    print 'Using clustering model %s...' % repr(cluster_model)
    print 'Clustering on training set to get codebook of %i words' % n_clusters

    # train kmeans or other cluster model on those descriptors selected above
    cluster_model.fit(all_train_descriptors)
    print 'done clustering.'

    return cluster_model

In [74]:
def histogram(cluster_model,des,K):
    words=cluster_model.predict(des)
    img_bow_hist = np.array([np.bincount(words, minlength=K)])
    return img_bow_hist

In [93]:
def cluster_split(des,Y, K, percent_test, percent_val):
    emo1=[]
    emo2=[]
    emo3=[]
    emo4=[]
    emo5=[]
    emo6=[]
    emo7=[]
    emotions=[1,2,3,4,5,6,7]
    
    print '\n total size of data:',len(des)
    
    # segregating database images based on emotion label
    for i,label in enumerate(Y):
        emo= str(label)
        a='emo'+emo
        des_label=[des_keypoint for des_keypoint in des[i]]
        locals()[a].append(des_label)
        
    #splitting data into test, train and validation such that every label is split equally
    train=[]
    Y_train=[]
    test=[]
    Y_test=[]
    validation=[]
    Y_val=[]
    for emo in emotions:
        total_rows=len(locals()['emo'+str(emo)])
        training_idxs, test_idxs, val_idxs = train_test_val_split_idxs(total_rows,percent_test,percent_val)
        train_emo=[locals()['emo'+str(emo)][i] for i in training_idxs]
        test_emo=[locals()['emo'+str(emo)][i] for i in test_idxs]
        val_emo=[locals()['emo'+str(emo)][i] for i in val_idxs]
        train_emo_label=[locals()['emo'+str(emo)][i] for i in training_idxs]
        test_emo=[locals()['emo'+str(emo)][i] for i in test_idxs]
        val_emo=[locals()['emo'+str(emo)][i] for i in val_idxs]
        for rows in train_emo:
            train.append(rows)
            Y_train.append(emo)
        for rows in test_emo:
            test.append(rows)
            Y_test.append(emo)
        for rows in val_emo:
            validation.append(rows)
            Y_val.append(emo)
    print '\n size of training data:',len(train),'\n size of validation data:',len(validation),'\n size of test data:',len(test)
    cluster_model = cluster_features(train,cluster_model=MiniBatchKMeans(n_clusters=K))
    
  
    print '\n calculting histogram vectors of training set'
    X_train=[]
    for des_img in train:
        hist=histogram(cluster_model,des_img,K)
        X_train.append(hist)
    X_train=np.array(X_train)
    X_train=X_train.reshape(len(X_train),K)
    
    # building validation set in terms of labels and vectors
    X_val=[]
    print '\n calculting histogram vectors of validation set'
    for des_img in validation:
        hist=histogram(cluster_model,des_img,K)
        X_val.append(hist)
    X_val=np.array(X_val)
    X_val=X_val.reshape(len(X_val),K)
    
    # building test set in terms of labels and vectors
    X_test=[]
    print '\n calculting histogram vectors of test set'
    for des_img in test:
        hist=histogram(cluster_model,des_img,K)
        X_test.append(hist)
    X_test=np.array(X_test)
    X_test=X_test.reshape(len(X_test),K)
    Y_train=np.array(Y_train)
    Y_val=np.array(Y_val)
    Y_test=np.array(Y_test)
    
    
    return X_train, Y_train, X_val, Y_val, X_test, Y_test, cluster_model

In [105]:
def run_svm(X_train, X_test, Y_train,Y_test,c_vals, gamma_vals,cv):
    param_grid = [{'C': c_vals, 'gamma': gamma_vals, 'kernel': ['rbf']}]
    svc = GridSearchCV(SVC(), param_grid,cv=cv, n_jobs=-1)
    svc.fit(X_train, Y_train)
    test_score = svc.score(X_test, Y_test)
    print 'test score :', test_score
    return svc, test_score

In [114]:
def svc_rbf(X_train,Y_train,skf,c,gam):
    svc=SVC(C=c, kernel='rbf',gamma= gam)
    print "SVM model initiated"
    score=[]
    for train,test in skf.split(X_train,Y_train):
        x=X_train[train]
        y=Y_train[train]
        svc.fit(x,y)
        scr_iter=svc.score(X_train[test],Y_train[test])
        score.append(scr_iter)
    return score

In [108]:
def vbov_grid():
    # initialising global variables
    path_data="/home/atrij/artificial_intelligence/CK+/Emotion"
    path_image="/home/atrij/artificial_intelligence/CK+/cohn-kanade-images"
    filename_des='img_des'
    filename_label='img_label'
    percent_test=0.1
    percent_val=0
    cv=10
    # checking for descriptor and label file in directory, if not present creating file
    if (os.path.exists(filename_des) & os.path.exists(filename_label))==0:
        ld= labeled_data(path_data,path_image)
        save_descriptor_label(filename_des,filename_label,ld)
    
    # load descriptors from img_des
    fileObject = open(filename_des,'r')  
    des = pickle.load(fileObject)
    fileObject.close()
      
    # load labels from img_des
    fileObject = open(filename_label,'r')
    Y = pickle.load(fileObject)
    fileObject.close()
    
    #scores = ['precision', 'recall']
    # retrieving image feature vectors using K-means clustering
    #cv = StratifiedKFold(n_splits=k_split)
    K=[200,225,250,275,300,325,350,375,400,425,450,475,500,525,550,575,600]
    c_vals=[1, 5, 10, 100]
    gamma_vals=[0.1, 0.01, 0.0001, 0.00001]
    for k in K:
        X_train, Y_train,_,_, X_test, Y_test,_= cluster_split(des,Y, k, percent_test, percent_val)
        svc, test_score = run_svm(X_train, X_test, Y_train,Y_test,c_vals,gamma_vals,cv)
        print svc.best_estimator_


In [107]:
vbov()


 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
60031 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=200, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 200 words
done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
test score : 0.535714285714
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1e-05, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
60144 descriptors bef

/home/atrij/.virtualenvs/cv/local/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1381: RuntimeWarning: init_size=300 should be larger than k=325. Setting it to 3*k
  init_size=init_size)


done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
test score : 0.607142857143
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
59470 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=350, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 350 words
done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting hi

/home/atrij/.virtualenvs/cv/local/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1381: RuntimeWarning: init_size=300 should be larger than k=375. Setting it to 3*k
  init_size=init_size)


done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
test score : 0.535714285714
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
60426 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=400, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 400 words


...

done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
test score : 0.5
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
60209 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=575, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 575 words


/home/atrij/.virtualenvs/cv/local/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1381: RuntimeWarning: init_size=300 should be larger than k=575. Setting it to 3*k
  init_size=init_size)


done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
test score : 0.571428571429
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
59902 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=600, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 600 words


/home/atrij/.virtualenvs/cv/local/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1381: RuntimeWarning: init_size=300 should be larger than k=600. Setting it to 3*k
  init_size=init_size)


done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
test score : 0.678571428571
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [117]:
def vbov_indi():
    # initialising global variables
    path_data="/home/atrij/artificial_intelligence/CK+/Emotion"
    path_image="/home/atrij/artificial_intelligence/CK+/cohn-kanade-images"
    filename_des='img_des'
    filename_label='img_label'
    percent_test=0.1
    percent_val=0
    cv=10
    # checking for descriptor and label file in directory, if not present creating file
    if (os.path.exists(filename_des) & os.path.exists(filename_label))==0:
        ld= labeled_data(path_data,path_image)
        save_descriptor_label(filename_des,filename_label,ld)
    
    # load descriptors from img_des
    fileObject = open(filename_des,'r')  
    des = pickle.load(fileObject)
    fileObject.close()
      
    # load labels from img_des
    fileObject = open(filename_label,'r')
    Y = pickle.load(fileObject)
    fileObject.close()
    
    #scores = ['precision', 'recall']
    # retrieving image feature vectors using K-means clustering
    skf = StratifiedKFold(n_splits=cv)
    K=[200,225,250,275,300,325,350,375,400,425,450,475,500,525,550,575,600]
    c_vals=[1, 5, 10, 100]
    gamma_vals=[0.1, 0.01, 0.0001, 0.00001]
     
    for k in K:
        X_train, Y_train,_,_, X_test, Y_test,_= cluster_split(des,Y, k, percent_test, percent_val)
        for c in c_vals:
            for gam in gamma_vals:
                score = svc_rbf(X_train,Y_train,skf,c,gam)
                avg=np.array(score).mean()
                print 'C:',c, 'gamma:',gam,'cv_score:', avg,'\n'


In [118]:
vbov_indi()


 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
60510 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=200, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 200 words
done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
SVM model initiated
C: 1 gamma: 0.1 cv_score: 0.248874351709 

SVM model initiated
C: 1 gamma: 0.01 cv_score: 0.397317556027 

SVM model initiated
C: 1 gamma: 0.0001 cv_score: 0.248874351709 

SVM model initiated
C: 1 gamma: 1e-05 cv_score: 0.248874351709 

SVM model initiated
C: 5 gamma: 0.1 cv_score: 0.248874351709 

SVM model initiated
C: 5 gamma: 0.01 cv_score: 0.4258209920

C: 100 gamma: 0.0001 cv_score: 0.576831323646 

SVM model initiated
C: 100 gamma: 1e-05 cv_score: 0.591219043207 


 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
59432 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=325, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 325 words
done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
SVM model initiated
C: 1 gamma: 0.1 cv_score: 0.248874351709 

SVM model initiated
C: 1 gamma: 0.01 cv_score: 0.426018357141 

SVM model initiated
C: 1 gamma: 0.0001 cv_score: 0.248874351709 

SVM model initiated
C: 1 gamma: 1e-05 cv_score: 0.248874351709 

SVM mod

C: 100 gamma: 0.01 cv_score: 0.431232234133 

SVM model initiated
C: 100 gamma: 0.0001 cv_score: 0.63047429565 

SVM model initiated
C: 100 gamma: 1e-05 cv_score: 0.588054329607 


 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
59983 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=450, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 450 words
done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
SVM model initiated
C: 1 gamma: 0.1 cv_score: 0.248874351709 

SVM model initiated
C: 1 gamma: 0.01 cv_score: 0.4230512824 

SVM model initiated
C: 1 gamma: 0.0001 cv_score: 0.248874351709 

SVM model

C: 100 gamma: 0.1 cv_score: 0.248874351709 

SVM model initiated
C: 100 gamma: 0.01 cv_score: 0.454105362553 

SVM model initiated
C: 100 gamma: 0.0001 cv_score: 0.589922450593 

SVM model initiated
C: 100 gamma: 1e-05 cv_score: 0.584517994904 


 total size of data: 326

 size of training data: 298 
 size of validation data: 0 
 size of test data: 28
60427 descriptors before clustering
Using clustering model MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=575, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)...
Clustering on training set to get codebook of 575 words
done clustering.

 calculting histogram vectors of training set

 calculting histogram vectors of validation set

 calculting histogram vectors of test set
SVM model initiated
C: 1 gamma: 0.1 cv_score: 0.248874351709 

SVM model initiated
C: 1 gamma: 0.01 cv_score: 0.429490469012 

SVM mod